In [1]:
import os 
from dataclasses import dataclass
from pathlib import Path
from ml_project import logger
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories
from sklearn.linear_model import ElasticNet
import joblib
import pandas as pd

In [2]:
os.chdir("../")
!pwd

f:/Tutorials/Deep Learning/Machine Learning Projects/End to End Machine Learning/machine-learning-with-mlflow


In [3]:
# Entity for the configuration from config.yaml

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_path: Path
    test_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [4]:
# Configuration for the ModelTrainer

class ModelTrainerConfirgurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH, schema_path = SCHEMA_FILE_PATH) -> None:
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.target_column

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_path = config.train_path,
            test_path = config.test_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        create_directories([config.root_dir])

        return model_trainer_config

In [5]:
# Create the component for ModelTrainer.

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_path)
        test_data = pd.read_csv(self.config.test_path)

        train_x = train_data.drop([self.config.target_column], axis = 1)
        test_x = test_data.drop([self.config.target_column], axis = 1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio = self.config.l1_ratio, random_state = 42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [6]:
# Define the pipline

try:
    config = ModelTrainerConfirgurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as error:
    raise error

[2023-08-06 16:21:25,161: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2023-08-06 16:21:25,164: INFO: common: yaml file: params.yaml loaded successfully.]
[2023-08-06 16:21:25,172: INFO: common: yaml file: schema.yaml loaded successfully.]
[2023-08-06 16:21:25,175: INFO: common: Created directory at: artifacts]
[2023-08-06 16:21:25,177: INFO: common: Created directory at: artifacts/model_trainer]
